* This code downloads S1 scenes for all S1 scenes whcih interesect the AOI on the target date.
* It creates chips of size 256x256 across the whole scene.

# Import libraries

In [1]:
from datetime import datetime
from pathlib import Path

import dateutil.rrule as rrule

from Methods import CoveringGrid as CoveringGrid, S1Downloader as S1Downloader

# Define variables

In [2]:
# path where the images will be downloaded to
outputPath = Path('.')/'Data'/'MyS1Project'

In [3]:
# projection and scale to be used for the output
projectCRS = 'epsg:3857'
scale = 10

In [4]:
# define chip size
chipSize = 256

In [5]:
# which date do we want to download images for, here a couple of examples are given of how to generate dates
# note that images are only donwloaded if they exist for this date

# single date
dates = ['2022-11-22']
# a series of dates
dates = ['2022-11-22', '2021-10-4', '2023-01-18']

#every day from start date to end date
startDate = '2022-01-01'
endDate = '2022-01-13'

dt = rrule.DAILY
dates = list(map(lambda date: date.strftime('%Y-%m-%d'), rrule.rrule(dt,
                      dtstart=datetime.strptime(startDate, '%Y-%m-%d'),
                      until=datetime.strptime(endDate, '%Y-%m-%d'))))

In [6]:
# path to the geojson containing the region of interest
pathToGeoJson = 'li.geojson'

In [7]:
# number of cores to use (parallel processing). Uses library 'multiprocess' if nbCores > 1
nbCores = 10

Here the folder structure is defined. The files are downloaded as follows:

baseFolder/<br>
├─ chipID/<br>
│  ├─ S1/<br>
│  │  ├─ chipID_date<br>
│  │  ├─ chipID_otherdate<br>
│  │  ├─ ...<br>
├─ otherChipID/<br>
│  ├─ s1/<br>
│  │  ├─ otherChipID_date<br>

You can change the folder structure below, you can use `chipID` and `date` as variables

In [8]:
chipPathLambda = lambda folder, chipID, date: folder/f'{chipID}'/'S1'/f'{chipID}_{date}.tif'

# Internatl setup

In [9]:
pathToGeoJson = Path(pathToGeoJson)
outputPathForCurrentGeoJson = outputPath/pathToGeoJson.stem
outputPathForCurrentGeoJson.mkdir(exist_ok=True, parents=True)

In [10]:
chipPathFunction = lambda chipID, date: chipPathLambda(outputPathForCurrentGeoJson, chipID, date)

# Download all images

In [11]:
# This loads the grid covering the study area
coveringGrid = CoveringGrid.GetCoveringGrid(outputPathForCurrentGeoJson, pathToGeoJson, projectCRS, scale, chipSize)

In [ ]:
for k, date in enumerate(dates):
    print(f'Processing date {date} ({k+1} / {len(dates)})')
    S1Downloader.DownloadAllChipsForDate(coveringGrid, date, projectCRS, chipSize, chipPathFunction, nbCores)

print(f'Done, everything downloaded to {outputPathForCurrentGeoJson}')